In [1]:
import pandas as pd
from tqdm import tqdm
from gensim.models import Word2Vec
import nltk
import numpy as np
import gensim
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/furiosa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Replace political party references with political party ids.

In [7]:
party_alts = {
'@μερα25':['Μ(έ|ε)ΡΑ25', 
           '((Μ(έ|ε)τωπο)|(Μετ(ώ|ω)που)) Ευρωπα(ϊ|ι)κ(ή|η)ς Ρεαλιστικ(ή|η)ς Ανυπακο(ή|η)ς', 
           'ΜΕΡΑ25'],
    
'@νδ':['Ν(έ|ε)α(ς*) Δημοκρατ(ί|ι)α(ς*)'], #' Ν.Δ.(?!\w)',
       #' ΝΔ(?!\w)'],  #look-ahead assertion: not followed by a word character: letter, number, underscore
    # ν.δ. νομοθετικό διάταγμα το γράφουν και με κεφαλαία
'η @νδ':['(Η|η) ΝΔ(?!\w)', '(Η|η) Ν.Δ.(?!\w)'], 
'της @νδ':['(Τ|τ)ης ΝΔ(?!\w)', '(Τ|τ)ης Ν.Δ.(?!\w)'],
'τη @νδ':['(Τ|τ)η ΝΔ(?!\w)', '(Τ|τ)η Ν.Δ.(?!\w)'],
    
#add space in front of small ids that could be wrongly identified in the middle of other words in caps lock
# space is not added also at the end of the string but look-ahead assertion that no word character follows.
    
' @δησυ':['Δημοκρατικ(ή|η)(ς*) Συμπαρ(ά|α)ταξη(ς*)', 
         ' ΔΗ\.ΣΥ\.(?!\w)', 
         ' ΔΗΣΥ(?!\w)'],
    
' @συριζα':['((Συνασπισμ(ό|ο)ς)|(Συνασπισμο(ύ|υ))) Ριζοσπαστικ(ή|η)ς Αριστερ(ά|α)ς', ' ΣΥΡΙΖΑ(?!\w)',' ΣΥ\.ΡΙΖ\.Α(?!\w)'],
# 'ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια':[],
    
'@ελληνικη_λυση':['Ελληνικ(ή|η)(ς*) Λ(ύ|υ)ση(ς*)((\s*)(\-*)(\s*)Κυρι(ά|α)κος Βελ(ό|ο)πουλος)?'],
                  #'Ελληνικ(ή|η)(ς*) Λ(ύ|υ)ση(ς*)'],
    # maybe just make 
' @πολαν':['Πολιτικ(ή|η)(ς*) (Ά|Α)νοιξη(ς*)', ' ΠΟΛ\.ΑΝ\.(?!\w)', ' ΠΟΛΑΝ(?!\w)'],
    
'@ανεξαρτητοι_δημοκρατικοι_βουλευτες':['Ανεξ(ά|α)ρτητοι Δημοκρατικο(ί|ι) Βουλευτ(έ|ε)ς', 
                                       'Ανεξ(ά|α)ρτητων Δημοκρατικ(ώ|ω)ν Βουλευτ(ώ|ω)ν', 
                                       'Ανεξ(ά|α)ρτητους Δημοκρατικο(ύ|υ)ς Βουλευτ(έ|ε)ς'],
    
' @ανελ':['Ανεξ(ά|α)ρτητοι (Έ|Ε)λληνες((\s*)(\-*)(\s*)Π(ά|α)νος Καμμ(έ|ε)νος)?', 
         'Ανεξ(ά|α)ρτητων Ελλ(ή|η)νων((\s*)(\-*)(\s*)Π(ά|α)νος Καμμ(έ|ε)νος)?', 
         'Ανεξ(ά|α)ρτητους (Έ|Ε)λληνες((\s*)(\-*)(\s*)Π(ά|α)νος Καμμ(έ|ε)νος)?', 
         ' ΑΝ.ΕΛ.(?!\w)', ' ΑΝΕΛ(?!\w)', 
         'Ανεξ(ά|α)ρτητοι (Έ|Ε)λληνες(\s*)(\-*)(\s*)Εθνικ(ή|η) Πατριωτικ(ή|η) Δημοκρατικ(ή|η) Συμμαχ(ί|ι)α'],
    
' @δηανα':['Δημοκρατικ(ή|η)(ς*) Αναν(έ|ε)ωση(ς*)', ' ΔΗΑΝΑ(?!\w)', ' ΔΗ\.ΑΝΑ\.(?!\w)'],
    
' @κιναλ':['Κ(ί|ι)νημα Αλλαγ(ή|η)ς', ' ΚΙΝΑΛ(?!\w)', ' ΚΙΝ\.ΑΛ\.(?!\w)', 'Κιν(ή|η)ματος Αλλαγ(ή|η)ς'],
    
' @δηκκι':['Δημοκρατικ(ό|ο) Κοινωνικ(ό|ο) Κ(ί|ι)νημα','Δημοκρατικο(ύ|υ) Κοινωνικο(ύ|υ) Κιν(ή|η)ματος', 
          ' ΔΗ\.Κ\.ΚΙ\.(?!\w)', ' ΔΗΚΚΙ(?!\w)'],
    
'@συνασπισμος':['Συνασπισμ(ό|ο|ός|ος|ού|ου) (τ|Τ)ης Αριστερ(ά|α)ς (τ|Τ)ων Κινημ(ά|α)των (κ|Κ)αι (τ|Τ)ης Οικολογ(ί|ι)ας', 
                'Συνασπισμ(ού|ου)(?!\s+Ριζοσπαστικ)',
                'Συνασπισμ(ός|ος|ό|ο)(?!\s+Ριζοσπαστικ)', 
                ' ΣΥΝ(?!\w)'], #with \s+ γιατί το ΣΥΝ ειναι συνηθες συνθετικο λεξεων?
    
' @πασοκ':['Πανελλ(ή|η)νιο Σοσιαλιστικ(ό|ο) Κ(ί|ι)νημα', 
          'Πανελλ(ή|η)νιου Σοσιαλιστικο(ύ|υ) Κιν(ή|η)ματος',
          ' ΠΑΣΟΚ(?!\w)', ' ΠΑ\.ΣΟ\.Κ\.(?!\w)'],
    
' @κκε':['Κομμουνιστικ(ό|ο) Κ(ό|ο)μμα Ελλ(ά|α)δ(ο|α)ς', 
        'Κομμουνιστικο(ύ|υ) Κ(ό|ο)μματος Ελλ(ά|α)δ(ο|α)ς',
        ' Κ\.Κ\.Ε\.(?!\w)', ' ΚΚΕ(?!\w)'],
    
' @λαος':['Λα(ϊ|ι)κ(ό|ο)(ς*) Ορθ(ό|ο)δοξο(ς*) Συναγερμ(ό|ο)(ς*)', 
         'Λα(ϊ|ι)κο(ύ|υ) Ορθ(ό|ο)δοξου Συναγερμο(ύ|υ)', 
         ' ΛΑ\.Ο\.Σ\.(?!\w)',' ΛΑΟΣ(?!\w)'], #not λαος because it is used for referring to people
    
' @χα':['Λα(ϊ|ι)κ(ό|ο)(ς*) Σ(ύ|υ)νδεσμο(ς*)(\s*)(\-*)(\s*)Χρυσ(ή|η) Αυγ(ή|η)', 
       'Λα(ϊ|ι)κο(ύ|υ) Συνδ(έ|ε)σμου(ς*)(\s*)(\-*)(\s*)Χρυσ(ή|η) Αυγ(ή|η)',
       'Χρυσ(ή|η)(ς*) Αυγ(ή|η)(ς*)', 
       ' Χ\.Α\.(?!\w)',],
    
' @οε':['Οικολ(ό|ο)γοι(\s*)(\-*)(\s*)Εναλλακτικο(ί|ι)', 
       'Οικολ(ό|ο)γων(\s*)(\-*)(\s*)Εναλλακτικ(ώ|ω)ν',
#        ' Ο\.Ε\.(?!\w)' ομορρυθμη εταιρεια συνηθως
       ],
    
' @λαε':['Λα(ϊ|ι)κ(ή|η)(ς*) Εν(ό|ο)τητα(ς*)', ' ΛΑ\.Ε\.(?!\w)', ' ΛΑΕ(?!\w)'],
    
'@ποταμι':['Ποτ(ά|α)μι'],
    
' @εκ':['(Έ|Ε)νωση(ς*) Κεντρ(ώ|ω)ων', 
#         ' Ε\.Κ\.(?!\w)' #αναφορα το 1989 σε κάτι άλλο με τα ίδια αρχικά
       ],
    
'@δημαρ':['Δημοκρατικ(ή|η)(ς*) Αριστερ(ά|α)(ς*)', ' ΔΗΜΑΡ(?!\w)', ' ΔΗΜ\.ΑΡ\.(?!\w)'],

}

def replace_political_parties_with_ids(text):
    
    text = re.sub('\s\s+' , ' ', text)

    for key_name, alt_names in party_alts.items():
        text = re.sub('|'.join(alt_names), key_name, text)
        
    text = re.sub('\s\s+' , ' ', text)
    
    return(text)

example = 'Της Δημοκρατικη Ανανέωση; Του Συνασπισμού και του ΠΑΣΟΚ.'
replace_political_parties_with_ids(example)
# df[df['speech'].str.contains("@")].speech.to_list()[:100]

'Της @δηανα; Του @συνασπισμος και του @πασοκ.'

In [ ]:
original_df = pd.read_csv('../out_files/tell_all_FILLED.csv')
original_df['speech'] = original_df['speech'].apply(replace_political_parties_with_ids)

In [8]:
df = original_df.copy()

Remove accents

In [9]:
import unicodedata
from tqdm import tqdm
import datetime

print(datetime.datetime.now())

tqdm.pandas()

def strip_accents(text):
    new_text =  ''.join(c for c in unicodedata.normalize('NFD', text)
                  if unicodedata.category(c) != 'Mn')
    if new_text=='':
        print(text)
    return(new_text)

df['speech'] = df.speech.progress_apply(lambda x: strip_accents(x))
print(df['speech'].iloc[0])

print(datetime.datetime.now())


/Users/furiosa/Documents/repos/paper_greekparliament/.parl_env/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  0%|          | 83/1280918 [00:00<26:33, 803.75it/s]

2021-03-12 14:50:14.280466


100%|██████████| 1280918/1280918 [11:42<00:00, 1823.89it/s]


 Παρακαλειται ο Γραμματεας κ. Βουλγαρακης να συνοδευσει το Μακαριοτατο Αρχιεπισκοπο Αθηνων και πασης Ελλαδος κ. ΣΕΡΑΦΕΙΜ και τα συνοδευοντα αυτον μελη της Ιερας Συνοδου κατα την εισοδο τους στην Αιθουσα της Βουλης, προκειμενου να τελεσθει αγιασμος. . Στη συνεχεια τελειται ο καθιερωμενος αγιασμος.
2021-03-12 15:01:58.683032


Replace stopwords with @sw

In [10]:
from spacy.lang.el.stop_words import STOP_WORDS #with accents

STOP_WORDS_no_accents = list(set([strip_accents(w.lower()) for w in STOP_WORDS]))

cschwem2er_stopwords = open('../out_files/greek_stopwords_cschwem2er_with_accents.txt', 'r+').read().split(',')

cschwem2er_stopwords_no_accents = list(set([strip_accents(w.lower()) for w in cschwem2er_stopwords]))
stopwords = list(set(STOP_WORDS_no_accents+cschwem2er_stopwords_no_accents))

# print(stopwords)
print(datetime.datetime.now())

remove = '|'.join(stopwords)

df['speech'] = df['speech'].str.lower()
print(df['speech'].iloc[0][:100])

df['speech'] = df['speech'].replace('@', ' ')

df['speech'] = df['speech'].replace(r'\b('+remove+r')\b', '@sw', regex=True) # (?i) for case insensitive search very slow
print(df['speech'].iloc[0][:100])
print(datetime.datetime.now())


2021-03-12 15:02:00.794531
 παρακαλειται ο γραμματεας κ. βουλγαρακης να συνοδευσει το μακαριοτατο αρχιεπισκοπο αθηνων και πασης
 παρακαλειται @sw γραμματεας κ. βουλγαρακης @sw συνοδευσει @sw μακαριοτατο αρχιεπισκοπο αθηνων @sw π
2021-03-12 15:39:18.340743


Remove punctuation: first tokenize text into sentences, then add space before and after punctuation to separate it from actual words, then tokenize sentences into lists of words, remove punctuation tokens and the empty strings and last remove empty sentences.

In [11]:
# from string import punctuation
import nltk
import re
import unicodedata

# use this regex to add space between the following symbols and text
punct_regex = re.compile(r'([±½°¶■≥§‰●$€!΄¨‘“"#%&\'()*+,-./:;<=>«»?[\]^_`{|}~\u00b7\u0387…–])')

#use this regex to remove the following characters from text
punctuation = '!"#%&\'()*+,-./:;<=>«»?[\]^_`{|}~\u00b7\u0387…–'
# symbols = '±½°¶■≥§‰●$€'

#ADD SPACE
def add_space(text):
    text = re.sub(punct_regex, r' \1 ', text)
    text = re.sub('\s{2,}', ' ', text)
    return text


#TOKENIZE SENTENCES
print('Tokenizing text into sentences...')
df.speech = df.speech.progress_apply(
    lambda x: nltk.sent_tokenize(x)) #list of lists

# ADD SPACE
print('Adding space between punctuation or other symbols...')
df.speech = df.speech.progress_apply(lambda x: [add_space(sentence) for sentence in x])


#TOKENIZE WORDS IN SENTENCES
print('Tokenizing sentences into lists of words...')

df.speech = df.speech.progress_apply(
    lambda x: [sentence.split() for sentence in x]) #list of lists

# remove punctuation and empty strings
print('Removing punctuation...')
df.speech = df.speech.progress_apply(lambda x: [[w for w in sent if (w not in punctuation and w!='')] for sent in x])

# Remove tokens with 2 or less characters
print('Removing tokens with <2 characters...')
df.speech = df.speech.progress_apply(lambda x: [[w for w in sent if ((not len(w)<=2) or ('@' in w) or (w.isdigit()))] for sent in x])

#Remove empty sentences
print('Removing empty sentences...')
df.speech = df.speech.progress_apply(lambda x: [sent for sent in x if len(sent)>0])

print('...done!')

  0%|          | 0/1280918 [00:00<?, ?it/s]

Tokenizing text into sentences...


  0%|          | 581/1280918 [00:00<03:40, 5809.02it/s]

Adding space between punctuation or other symbols...


  0%|          | 2467/1280918 [00:00<00:51, 24660.15it/s]

...

Removing tokens with <2 characters...


  0%|          | 0/1280918 [00:00<?, ?it/s]

Removing empty sentences...


100%|██████████| 1280918/1280918 [04:33<00:00, 4686.51it/s]  


...done!


In [12]:
df.head(2)

,member_name,sitting_date,parliamentary_period,parliamentary_session,parliamentary_sitting,political_party,government,member_region,roles,member_gender,speaker_info,speech
0,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,προεδρευων,"[[παρακαλειται, @sw, γραμματεας, βουλγαρακης, ..."
1,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,προεδρευων,"[[παρακαλειται, @sw, κυριος, γραμματεας, @sw, ..."


In [13]:
df.speech = df.speech.apply(lambda x: [' '.join([token for token in sentence]) for sentence in x])
df.speech = df.speech.apply(lambda x: '.'.join([sentence for sentence in x]))
df.to_csv('../out_files/tell_all_cleaned.csv', encoding='utf-8', index=False, na_rep=np.nan) #, encoding='utf-8', index=False, na_rep=np.nan
